# Inspección de variables y el espacio de trabajo

(Notas basadas en el libro **Learning R de Richard Cotton**)

Hasta ahora, hemos realizado algunos cálculos y asignado algunas variables. En este cuaderno, encontraremos formas de examinar las propiedades de esas variables y manipular el espacio de trabajo del usuario que las contiene.

## Clases 


Todas las variables en R tienen una clase, que te dice qué tipos de variables son. Por ejemplo, la mayoría de los números tienen clase `numeric`  y los valores lógicos tienen la clase `logic`.  En realidad, los `vectores` de números son `numric` y los `vectores` de valores lógicos son `logic`, ya que R no tiene tipos escalares. El tipo de datos *más pequeño* en R es un vector.

Podemos averiguar la  clase de una variable utilizando `class(variable)`:

In [1]:
class(c(TRUE, FALSE))

[1] "logical"

Vale la pena ser consciente de que, al igual que una clase, todas las variables también tienen un tipo de almacenamiento interno (accedido a través de `typeof`), un modo (`mode`) y un modo de almacenamiento (`storage.mode`). Los tipos, modos y modos de almacenamiento existen en su mayoría para propósitos heredados, por lo que en la práctica sólo necesitarás usar `class` de un objeto.

In [2]:
set.seed(2)
x <- 1:10
typeof(x)

y <- x/5 + rnorm(10)
typeof(y)

g <- lm(y ~ x)
typeof(g)


[1] "integer"

[1] "double"

[1] "list"

### Diferentes tipos de números

Todas las variables que creamos  eran números, pero R contiene tres clases diferentes de variable numérica: `numeric` para valores en coma flotante, `integer ` para los  enteros y `complex` para los números complejos. Podemos decir cuál es cuál examinando la clase de la variable:

In [3]:
class(sqrt(1:10))

[1] "numeric"

In [4]:
class(1 + 2i)

[1] "complex"

In [5]:
class(4L) # L hace que el numero sea un entero

[1] "integer"

In [6]:
class(0.5:4.5)

[1] "numeric"

Tenga en cuenta que a la hora de escribir, todos los números de punto flotante son números de `32` bits (*doble precisión*), incluso cuando se instala en un sistema operativo de 64 bits. 

Escribiendo `.Machine` te da cierta información sobre las propiedades de los números de R. Aunque los valores, en teoría, pueden cambiar de máquina a máquina, para la mayoría de las construcciones, la mayoría de los valores son los mismos. Muchos de los valores devueltos por `.Machine` nunca deben preocuparte. Vale la pena saber que el número de punto flotante más grande que R puede representar con precisión completa es de aproximadamente `1.8e308`. Esto es bastante grande para  propósitos diarios, pero mucho más pequeño que el infinito.  El número positivo más pequeño que se puede representar es `2.2e-308`. Los números enteros pueden tomar valores hasta `2^31 - 1`, que es un poco más de dos mil millones y el menor `-2^31 - 1`.

In [1]:
.Machine 

$double.eps
[1] 2.220446e-16

$double.neg.eps
[1] 1.110223e-16

$double.xmin
[1] 2.225074e-308

$double.xmax
[1] 1.797693e+308

$double.base
[1] 2

$double.digits
[1] 53

$double.rounding
[1] 5

$double.guard
[1] 0

$double.ulp.digits
[1] -52

$double.neg.ulp.digits
[1] -53

$double.exponent
[1] 11

$double.min.exp
[1] -1022

$double.max.exp
[1] 1024

$integer.max
[1] 2147483647

$sizeof.long
[1] 8

$sizeof.longlong
[1] 8

$sizeof.longdouble
[1] 16

$sizeof.pointer
[1] 8

El único otro valor de gran interés es $\epsilon$, el número de coma flotante positivo más pequeño tal que $\vert \epsilon + 1\vert != 1$. Esa es una manera elegante de decir lo cerca que dos números pueden estar para que R sepa que son diferentes. Se trata de `2.2e-16`. Este valor es utilizado por `all.equal` cuando se comparan dos vectores numéricos.


In [8]:
# El epsilon de la maquina puede ser obtenido

macheps <- function(){
  eps =1
  while(1 + eps/2!= 1)eps = eps/2
  eps
}
macheps()

[1] 2.220446e-16

## Otras clases

Además de las tres clases numéricas y la clase lógica que hemos visto, hay tres clases más de vectores: `caracter` para almacenar el texto, `factors` para almacenar datos categóricos y `raw` para almacenar datos binarios.

En este próximo ejemplo, creamos un vector de caracteres utilizando el operador `c`, al igual que lo hicimos con los vectores numéricos. La clase de un vector de caracteres es `character`:


In [9]:
class(c("ella", "es", "bella", "ella", "es", "una", "estrella"))

[1] "character"

Ten  en cuenta que, a diferencia de algunos lenguajes, R no distingue entre cadenas y caracteres individuales, una cadena que contiene un carácter se trata igual que cualquier otra cadena. A diferencia de otros lenguajes de bajo nivel no es necesario preocuparse por la terminación de las cadenas en un carácter nulo `(\0)`. De hecho, es un error intentar incluir tal carácter en las cadenas.


En muchos lenguajes de programación, los datos categóricos estarían representados por números enteros. Por ejemplo,  `genero` podría representarse como `1` para las mujeres y `2` para los hombres. Una solución ligeramente mejor sería tratar  `genero` como una variable de carácter con las elecciones *mujer* y *hombre *. Sin embargo, esto sigue siendo semánticamente bastante dudoso, ya que los datos categóricos son un concepto diferente del texto plano. R tiene una solución más sofisticada que combina ambas ideas en una clase semánticamente correcta-los factores son números enteros con etiquetas:

In [10]:
(genero <- factor(c("masculino", "femenino", "femenino", "masculino", "femenino")))

[1] masculino femenino  femenino  masculino femenino 
Levels: femenino masculino

In [11]:
levels(genero)

[1] "femenino"  "masculino"

El contenido del factor se parece mucho a su equivalente de caracteres: obtiene etiquetas legibles para cada valor. Esas etiquetas se limitan a valores específicos (en este caso *femenino* y *masculino*) conocidos como los niveles del factor:

In [12]:
nlevels(genero)

[1] 2

Observa  que aunque *masculino*  es el primer valor en el `genero`, el primer nivel es *femenino*.  Por defecto, los niveles de factor se asignan alfabéticamente. A escondidas, los valores de factor se almacenan como números enteros en lugar de caracteres. Podemos ver esto más claramente usando `as.integer`:

In [13]:
as.integer(genero)

[1] 2 1 1 2 1

Este uso de números enteros para el almacenamiento  de los valores de los factores los hace muy eficientes en comparación con los  caracteres de texto, al menos cuando hay un montón de cadenas repetidas, como aquí. En el código siguiente, `sample` devuelve un vector de caracteres que convertimos en un factor utilizando `as.factor` y `object.size` devuelve la asignación de memoria para cada objeto:

In [14]:
genero_caracter <- sample(c("femenino", "masculino"), 10000, replace = TRUE)
genero_factor <- as.factor(genero_caracter)
object.size(genero_caracter)

80152 bytes

In [15]:
object.size(genero_factor)

40528 bytes

La clase `raw` almacena vectores de bytes *raw*.  Cada byte está representado por un valor hexadecimal de dos dígitos. Estos se  utilizan principalmente para almacenar el contenido de archivos binarios importados y como tales, son razonablemente raros. Los números enteros `0` a `255` se pueden convertir en raw utilizando `as.raw`. Las partes fraccionarias e imaginarias son descartadas y los números fuera de este rango se tratan como `0`. Para las cadenas, `as.raw` no funciona, en este caso se debe  usar `charToRaw` en su lugar:

In [16]:
as.raw(1:17)

 [1] 01 02 03 04 05 06 07 08 09 0a 0b 0c 0d 0e 0f 10 11

In [17]:
as.raw(c(pi, 1 + 1i, -1, 234))

Warning message in eval(expr, envir, enclos):
“partes imaginarias descartadas en la coerción”Warning message in eval(expr, envir, enclos):
“valores fuera de rango tratados como 0 en la coerción a crudo (raw)”

[1] 03 01 00 ea

In [18]:
(sushi <- charToRaw("Pescado!"))

[1] 50 65 73 63 61 64 6f 21

In [19]:
class(sushi)

[1] "raw"

Además de las clases de vectores que hemos visto hasta ahora, existen muchos otros tipos de variables, por ejemplo las matrices contienen datos multidimensionales, y las matrices con la clase `matrix` son un caso especial de matrices bidimensionales.

In [20]:
x <- matrix( c(6,7), nrow=2 )
class(x)

[1] "matrix"

### Comprobación y cambio de clases 

Llamar a la función `class` es útil para examinar interactivamente nuestras variables en el prompt de comandos, pero si queremos probar el tipo de un objeto en nuestros códigos, es mejor usar la función `is` o una de sus variantes específicas de clase. En una situación típica, nuestra prueba se verá como:

```
if(!is(x, "alguna_clase"))
{
   ...
}
```
La mayoría de las clases comunes tienen sus propias funciones `is.*` y  llamarlas normalmente es un poco más eficiente que usar la función general `is`. Por ejemplo:

In [21]:
is.character("rojo verde, amarillo fresa")

[1] TRUE

In [22]:
is.logical(FALSE)

[1] TRUE

In [23]:
is.list(list(a = 1, b = 2))

[1] TRUE

Podemos ver una lista completa de todas las funciones `is` del paquete base usando:

In [24]:
ls(pattern = "^is", baseenv())

[1] "is.array"                "is.atomic"              
 [3] "is.call"                 "is.character"           
 [5] "is.complex"              "is.data.frame"          
 [7] "is.double"               "is.element"             
 [9] "is.environment"          "is.expression"          
[11] "is.factor"               "is.finite"              
[13] "is.function"             "is.infinite"            
[15] "is.integer"              "is.language"            
[17] "is.list"                 "is.loaded"              
[19] "is.logical"              "is.matrix"              
[21] "is.na"                   "is.na.data.frame"       
[23] "is.na.numeric_version"   "is.na.POSIXlt"          
[25] "is.na<-"                 "is.na<-.default"        
[27] "is.na<-.factor"          "is.na<-.numeric_version"
[29] "is.name"                 "is.nan"                 
[31] "is.null"                 "is.numeric"             
[33] "is.numeric_version"      "is.numeric.Date"        
[35] "is.numeric.difftime"     "is.numeric.POSIXt"      
[37] "is.object"               "is.ordered"             
[39] "is.package_version"      "is.pairlist"            
[41] "is.primitive"            "is.qr"                  
[43] "is.R"                    "is.raw"                 
[45] "is.recursive"            "is.single"              
[47] "is.symbol"               "is.table"               
[49] "is.unsorted"             "is.vector"              
[51] "isatty"                  "isBaseNamespace"        
[53] "isdebugged"              "isIncomplete"           
[55] "isNamespace"             "isNamespaceLoaded"      
[57] "isOpen"                  "isRestart"              
[59] "isS4"                    "isSeekable"             
[61] "isSymmetric"             "isSymmetric.matrix"     
[63] "isTRUE"

Una rareza es que `is.numeric` devuelve `TRUE` para enteros así como valores de punto flotante. Si queremos probar sólo números de coma flotante, entonces debemos usar `is.double`. Sin embargo, esto no suele ser necesario, ya que R está diseñado para que los valores de punto flotante y enteros se puedan usar de forma más o menos intercambiable. En los siguientes ejemplos, tenga en cuenta que al añadir el sufijo L el número se convierte en un entero:

In [25]:
is.numeric(3)

[1] TRUE

In [26]:
is.numeric(3L)

[1] TRUE

In [27]:
is.integer(3)

[1] FALSE

In [28]:
is.integer(3L)

[1] TRUE

In [29]:
is.double(4)

[1] TRUE

In [30]:
is.double(4L)

[1] FALSE

A veces deseamos cambiar el tipo de un objeto. Esto se llama `casting`  y la mayoría de  funciones `is*` tienen una función correspondiente `as*` para lograrlo. Las funciones especializadas como `as*` deben  ser usadas sobre `as`  cuando estén disponibles, ya que son generalmente más eficientes y a menudo contienen una lógica adicional específica para cada clase. Por ejemplo, al convertir una cadena a un número, `as.numeric` es ligeramente más eficiente que `as`, pero cualquiera de ellas puede ser utilizada:

In [31]:
x <- "123.456"
as(x, "numeric")

[1] 123.456

In [32]:
as.numeric(x)

[1] 123.456

En este  ejemplo, sin embargo, el convertir un vector en un data frame, la función `as` lanza un error:

In [33]:
y <- c(2, 12, 343, 34997)
as(y, "data.frame")
as.data.frame(y)

ERROR: Error in as(y, "data.frame"): no method or default for coercing “numeric” to “data.frame”


## Examinando Variables

Siempre que hayamos hecho un cálculo o el nombre de una variable en la consola de R, el resultado se imprime. Esto sucede porque R llama implícitamente al método `print` del objeto.

Por lo tanto, escribir `1 + 1` en la consola de R, es lo mismo que `print(1 + 1)`.

Dentro de bucles o funciones, la impresión automática no ocurre, por lo que tenemos que llamar explícitamente a `print`:

In [34]:
ulams_spiral <- c(1, 8, 23, 46, 77)
for(i in ulams_spiral) i # estos valores no se imprimen
for(i in ulams_spiral) print(i)

[1] 1
[1] 8
[1] 23
[1] 46
[1] 77


Esto también es cierto en algunos sistemas se ejecuta R desde un terminal en lugar de usar una `GUI` o un  `IDE`. En este caso, siempre tendrá que llamar explícitamente a la función `print`.

La mayoría de las funciones de impresión se basan en llamadas a la función  de bajo nivel `cat`. Casi nunca tendrás que llamar a `cat` directamente (`print ` y `message` son  equivalentes a nivel de usuario), pero vale la pena saber  en caso de que necesites escribir tu propia función `print`.

Además de ver la impresión de una variable, a menudo es útil ver algún tipo de resumen del objeto. La función `summary` hace exactamente eso, dando la información apropiada para diferentes tipos de datos. Las variables numéricas se resumen como media, mediana y algunos cuantiles. Aquí, la función `runif` genera 30 números aleatorios que están uniformemente distribuidos entre `0` y `1`:

In [35]:
num <- runif(30)
summary(num)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
 0.0157  0.2374  0.4269  0.4493  0.6273  0.9978 

Los vectores categóricos y lógicos se resumen por los conteos de cada valor. En este ejemplo siguiente, `letters` es una constante incorporada que contiene los valores en minúsculas de `a` a la `z` (`LETTERS` contiene los equivalentes en mayúsculas, `A` a `Z`). Aquí, `letters[1: 8]` usan la indexación para restringir las letras a `a` a `h`. La función  `sample` realiza un muestreo aleatorio de estos valores, con `replace`, `30` veces:

In [36]:
fac <- factor(sample(letters[1:8], 30, replace = TRUE))
summary(fac)

a b c d e f g h 
3 2 4 1 4 4 4 8

In [37]:
bool <- sample(c(TRUE, FALSE, NA), 30, replace = TRUE)
summary(bool)

   Mode   FALSE    TRUE    NA's 
logical       9       6      15 

Los objetos multidimensionales, como matrices y data frames, se resumen en columnas. En un  data frame  `dfr` de `30` filas y objetos más  grandes es necesario reducir la información, usando la función `head` que  se puede utilizar para mostrar sólo las primeras filas (seis por defecto):

In [38]:
dfr <- data.frame(num, fac, bool)
head(dfr)

num,fac,bool
0.6419013,d,FALSE
0.1282398,c,FALSE
0.3619310,h,FALSE
0.9860313,h,NA
0.9977651,g,NA
0.7575694,h,NA


La función `summary` para data frames  funciona como  cuando  llamamos a  `summary `en cada columna individual:

In [39]:
summary(dfr)

      num              fac       bool        
 Min.   :0.0157   h      :8   Mode :logical  
 1st Qu.:0.2374   c      :4   FALSE:9        
 Median :0.4269   e      :4   TRUE :6        
 Mean   :0.4493   f      :4   NA's :15       
 3rd Qu.:0.6273   g      :4                  
 Max.   :0.9978   a      :3                  
                  (Other):3                  

Del mismo modo, la función `str` muestra la estructura del objeto. No es tan interesante para los vectores, pero `str` es muy útil para los data frames y listas anidadas:

In [40]:
str(num)

 num [1:30] 0.642 0.128 0.362 0.986 0.998 ...


In [41]:
str(dfr)

'data.frame':	30 obs. of  3 variables:
 $ num : num  0.642 0.128 0.362 0.986 0.998 ...
 $ fac : Factor w/ 8 levels "a","b","c","d",..: 4 3 8 8 7 8 7 3 6 1 ...
 $ bool: logi  FALSE FALSE FALSE NA NA NA ...


Cada clase  tiene su propio método `print` que controla cómo se muestra en la consola. A veces esta impresión oscurece su estructura interna o omite información útil. La función `unclass` se puede utilizar para evitarlo, permitiendo ver cómo se construye una variable. Por ejemplo, llamar a `unclass` en un factor revela que es sólo un vector entero, con un atributo llamado `levels`:

In [43]:
help(unclass)
unclass(fac)

[1] 4 3 8 8 7 8 7 3 6 1 8 1 5 7 5 5 7 8 1 8 2 3 5 8 3 8 6 2 6 6
attr(,"levels")
[1] "a" "b" "c" "d" "e" "f" "g" "h"

Es útil saber que la función  `attributes` proporciona una lista de todos los atributos que pertenecen a un objeto:

In [44]:
attributes(fac)

$levels
[1] "a" "b" "c" "d" "e" "f" "g" "h"

$class
[1] "factor"

In [46]:
# Ejemplo 1

a <- matrix(1:4, nr=2)
rownames(a) <- letters[1:2]
colnames(a) <- LETTERS[1:2]

str(a)

 int [1:2, 1:2] 1 2 3 4
 - attr(*, "dimnames")=List of 2
  ..$ : chr [1:2] "a" "b"
  ..$ : chr [1:2] "A" "B"


In [47]:
unclass(a)

,A,B
a,1,3
b,2,4


In [48]:
attributes(a)

$dim
[1] 2 2

$dimnames
$dimnames[[1]]
[1] "a" "b"

$dimnames[[2]]
[1] "A" "B"

In [49]:
# Ejemplo 2

data(USArrests)
r <- princomp(USArrests)$loadings

class(r)

[1] "loadings"

In [50]:
unclass(r)

,Comp.1,Comp.2,Comp.3,Comp.4
Murder,-0.04170432,0.04482166,0.07989066,0.99492173
Assault,-0.99522128,0.05876003,-0.06756974,-0.03893830
UrbanPop,-0.04633575,-0.97685748,-0.20054629,0.05816914
Rape,-0.07515550,-0.20071807,0.97408059,-0.07232502


In [51]:
print.default(r)

              Comp.1      Comp.2      Comp.3      Comp.4
Murder   -0.04170432  0.04482166  0.07989066  0.99492173
Assault  -0.99522128  0.05876003 -0.06756974 -0.03893830
UrbanPop -0.04633575 -0.97685748 -0.20054629  0.05816914
Rape     -0.07515550 -0.20071807  0.97408059 -0.07232502
attr(,"class")
[1] "loadings"


In [52]:
 str(r)

 loadings [1:4, 1:4] -0.0417 -0.9952 -0.0463 -0.0752 0.0448 ...
 - attr(*, "dimnames")=List of 2
  ..$ : chr [1:4] "Murder" "Assault" "UrbanPop" "Rape"
  ..$ : chr [1:4] "Comp.1" "Comp.2" "Comp.3" "Comp.4"


In [53]:
# Ejemplo 3

n <- 200
x <- rnorm(n)
y <- 1 - 2 * x + rnorm(n)
r1 <- lm(y~x)
r2 <- summary(r1)

In [54]:
str(r1)

List of 12
 $ coefficients : Named num [1:2] 1.02 -2
  ..- attr(*, "names")= chr [1:2] "(Intercept)" "x"
 $ residuals    : Named num [1:200] -0.219 0.861 0.418 -1.779 1.063 ...
  ..- attr(*, "names")= chr [1:200] "1" "2" "3" "4" ...
 $ effects      : Named num [1:200] -14.9 -25.655 0.479 -1.765 1.056 ...
  ..- attr(*, "names")= chr [1:200] "(Intercept)" "x" "" "" ...
 $ rank         : int 2
 $ fitted.values: Named num [1:200] -0.415 1.806 -0.445 0.985 1.599 ...
  ..- attr(*, "names")= chr [1:200] "1" "2" "3" "4" ...
 $ assign       : int [1:2] 0 1
 $ qr           :List of 5
  ..$ qr   : num [1:200, 1:2] -14.1421 0.0707 0.0707 0.0707 0.0707 ...
  .. ..- attr(*, "dimnames")=List of 2
  .. .. ..$ : chr [1:200] "1" "2" "3" "4" ...
  .. .. ..$ : chr [1:2] "(Intercept)" "x"
  .. ..- attr(*, "assign")= int [1:2] 0 1
  ..$ qraux: num [1:2] 1.07 1.03
  ..$ pivot: int [1:2] 1 2
  ..$ tol  : num 1e-07
  ..$ rank : int 2
  ..- attr(*, "class")= chr "qr"
 $ df.residual  : int 198
 $ xlevels      : 

In [56]:
unclass(r1)

$coefficients
(Intercept)           x 
   1.015937   -2.004828 

$residuals
           1            2            3            4            5            6 
-0.218604684  0.860878242  0.418191841 -1.778879429  1.062722941  0.048911370 
           7            8            9           10           11           12 
-0.002063298 -0.987539743  1.025877152 -1.565124671  0.289392362  0.262456364 
          13           14           15           16           17           18 
 2.171501312 -2.147579902  1.954696715 -0.906116139  0.803979598 -1.073892627 
          19           20           21           22           23           24 
-0.505419194 -1.970814326 -0.609269939  0.409610390 -0.219459152  0.287701913 
          25           26           27           28           29           30 
 0.179305465 -1.541541739  0.289142909  0.433402154  0.198240179 -0.376572118 
          31           32           33           34           35           36 
 1.261610565  0.047974499 -1.452631640 -0.601391173  0.

## El espacio de trabajo (Workspace)

Mientras trabajamos, a menudo es bueno saber los nombres de las variables que hemos creado y lo que contienen. Para enumerar los nombres de las variables existentes, utiliza la función `ls`. Este nombre lleva el nombre del comando Unix equivalente y sigue la misma convención: por defecto, los nombres de las variables que comienzan con un `.` están escondidos. Para verlos, pase el argumento `all.names = TRUE`:

In [57]:
r <- 1
b <- "Jessica"
c <- TRUE
ls()

[1] "a"               "b"               "bool"            "c"              
 [5] "dfr"             "fac"             "g"               "genero"         
 [9] "genero_caracter" "genero_factor"   "i"               "macheps"        
[13] "n"               "num"             "r"               "r1"             
[17] "r2"              "sushi"           "ulams_spiral"    "USArrests"      
[21] "x"               "y"

In [58]:
ls(pattern = "ac")

[1] "fac"             "genero_caracter" "genero_factor"   "macheps"

Para más información sobre nuestro espacio de trabajo, podemos ver la estructura de nuestras variables usando `ls.str`. Esto es, como es de esperar, una combinación de las funciones `ls` y `str` y es muy útil durante las sesiones de depuración. `browseEnv` proporciona una capacidad similar, pero muestra su salida en una página HTML en nuestro navegador web:

In [59]:
browseEnv()

R objects in .GlobalEnv environment is shown in browser ‘/usr/bin/firefox’ 


Después de trabajar durante un tiempo, especialmente durante la exploración de datos, nuestro espacio de trabajo puede llegar a ser bastante desordenado. Podemos limpiarlo usando la función `rm ` para eliminar variables:

In [60]:
rm(r, c)
# rm(list = ls()) 

In [61]:
ls()

[1] "a"               "b"               "bool"            "dfr"            
 [5] "fac"             "g"               "genero"          "genero_caracter"
 [9] "genero_factor"   "i"               "macheps"         "n"              
[13] "num"             "r1"              "r2"              "sushi"          
[17] "ulams_spiral"    "USArrests"       "x"               "y"